# Quickstart

In [26]:
from foxflow.reader import BagfileReader
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Foxglove API key (set in .env)
FOXGLOVE_KEY = os.getenv("FOXGLOVE_KEY")

In [3]:
r = BagfileReader(FOXGLOVE_KEY)
r.print_recordings()

Available Recordings:

bc_pitstop                               → rec_0dyFsZgY0g4uRecO
bc_hm_right                              → rec_0dyG6uEXMKleoJ1z
bc_hm_left                               → rec_0dyG7RYeZx4Zmyiw
runde_vdi_lausitz                        → rec_0dyVbk9iNjsByd1Z
vdi_adc_2025_left                        → rec_0dyVch3zRMAviL5U
vdi_adc_2025_right                       → rec_0dyVcoxClnfxzloX
szechenyi_bc_ir_60_left_0                → rec_0e02rpVN69OXAheD
szechenyi_bc_ir_90_left_0                → rec_0e02rqfrxfbW3xKy
szechenyi_bc_uss_left                    → rec_0e02rcEt7K9TwwR4
testfile                                 → rec_0e4fjnBMpmFLl9Np
bc_acc_left_I                            → rec_0dxlVpedvsMyiMmM
bc_acc_left_lidar                        → rec_0dxlWSHulsX1fuoR
bc_acc_right_I                           → rec_0dxlWmQlvwikncqG
bc_acc_right_lidar                       → rec_0dxlYIDbBRQIpBFt
bc_left_vol_II                           → rec_0dxlYUL1DQxE1MGg
bc_left_vol_I    

In [21]:
df_info = r.select_recording_by_name('parking_spot_scan_boxes')
display(df_info)

,topic,version,encoding,schema_encoding,schema_name
0,/camera/camera/imu,8e809c37c92f0e8aae01bf2534f5b99f,cdr,ros2msg,sensor_msgs/msg/Imu
1,/imu,8e809c37c92f0e8aae01bf2534f5b99f,cdr,ros2msg,sensor_msgs/msg/Imu
2,/rc/ackermann_cmd,e31eb8800d99f34ad9cf4113bc497ad8,cdr,ros2msg,ackermann_msgs/msg/AckermannDriveStamped
3,/rc/joy,6f953c4447ef12d50356d344b4aceaf9,cdr,ros2msg,sensor_msgs/msg/Joy
4,/scan,063576fdeec8e68b5e23cf97c3adbc85,cdr,ros2msg,sensor_msgs/msg/LaserScan
5,/tf_static,a7d722ef103066448164633941760d88,cdr,ros2msg,tf2_msgs/msg/TFMessage
6,/uss_sensors,344f32583ef9726975feb8c367f334c8,cdr,ros2msg,std_msgs/msg/Int16MultiArray


#### Laser Scan

In [24]:
df_scan = r.read_topic('/scan')[0]
df_scan.head()

,timestamp_ns,angle_min,angle_max,angle_increment,time_increment,scan_time,range_min,range_max,ranges,intensities
0,946690865515522816,-3.124139,3.141593,0.003483,0.000049,0.088427,0.15,16.0,"[0.9120000004768372, 0.9120000004768372, 0.913...","[0.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0..."
1,946690865605088128,-3.124139,3.141593,0.003483,0.000051,0.091301,0.15,16.0,"[0.9100000262260437, 0.9120000004768372, 0.912...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."
2,946690865697576448,-3.124139,3.141593,0.003483,0.000050,0.090358,0.15,16.0,"[0.9100000262260437, 0.9100000262260437, 0.910...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."
3,946690865789197056,-3.124139,3.141593,0.003483,0.000054,0.097363,0.15,16.0,"[0.9120000004768372, 0.9120000004768372, 0.910...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."
4,946690865887939712,-3.124139,3.141593,0.003483,0.000051,0.090873,0.15,16.0,"[0.9100000262260437, 0.9079999923706055, 0.906...","[47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47.0, 47...."


In [33]:
row = df_scan.iloc[0]

# ranges
r = np.array(row["ranges"])

# angles (rad → deg)
theta = np.degrees(
    np.linspace(
        row["angle_min"],
        row["angle_max"],
        num=len(r),
        endpoint=False
    )
)

# polar scatter with range as color
fig = px.scatter_polar(
    r=r,
    theta=theta,
    color=r,                 # distance → color
    color_continuous_scale="Viridis",
    labels={"color": "range [m]"},
)

fig.show()


#### Events

In [20]:
# Fetch events (list of dicts)
events = r.get_events()

# Flatten nested dicts into a table
df = pd.json_normalize(events)

# Simple timeline: one bar per event
fig = px.timeline(
    df,
    x_start="start",   # event start time
    x_end="end",       # event end time
    y="id",            # one row per event
    hover_data=["id"], # minimal hover info
)

# Display plot
fig.show()
